 - Local Documents - txt/pdf/csv etc
 - 

In [6]:

# ! pip install -U -q google.generativeai chromadb langchain langchain-google-genai python-dotenv

In [7]:
import os

In [8]:

DATA_PATH = os.path.join('..', 'data')
FILENAME = 'yolo.pdf'

FAISS_VECTORDB_PATH = './vectordb/faissdb'
CHROMA_VECTORDB_PATH = './vectordb/chromadb'

MODEL = "mistral" # mistral/gemini
VECTORDB = "faiss" # faiss/chroma

In [9]:
## Google API Key
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] =GOOGLE_API_KEY

## Retriever
- Read the documents form the local 
- Create the embeddings
- Save the embeddings in the vectorDB - chromadb
- Create a retriever object

In [10]:
## Load the documents

from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.document_loaders import UnstructuredFileLoader


document_path = os.path.join(DATA_PATH, FILENAME)


# loader = UnstructuredFileLoader(os.path.join(document_path, 'yolo.pdf'))


loader = PyPDFLoader(document_path)
docs = loader.load()
docs[0]


Document(page_content='You Only Look Once:\nUniﬁed, Real-Time Object Detection\nJoseph Redmon∗, Santosh Divvala∗†, Ross Girshick¶, Ali Farhadi∗†\nUniversity of Washington∗, Allen Institute for AI†, Facebook AI Research¶\nhttp://pjreddie.com/yolo/\nAbstract\nWe present YOLO, a new approach to object detection.\nPrior work on object detection repurposes classiﬁers to per-\nform detection. Instead, we frame object detection as a re-\ngression problem to spatially separated bounding boxes and\nassociated class probabilities. A single neural network pre-\ndicts bounding boxes and class probabilities directly from\nfull images in one evaluation. Since the whole detection\npipeline is a single network, it can be optimized end-to-end\ndirectly on detection performance.\nOur uniﬁed architecture is extremely fast. Our base\nYOLO model processes images in real-time at 45 frames\nper second. A smaller version of the network, Fast YOLO,\nprocesses an astounding 155 frames per second while\nstill ac

In [11]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

## get page content
document_text = docs[0].page_content

recursive_char_text_splitter=RecursiveCharacterTextSplitter(
                                                chunk_size=500,
                                                chunk_overlap=50)
documents=recursive_char_text_splitter.split_documents(docs)

# Convert the text to LangChain's `Document` format
# docs =  [Document(page_content=document_text, metadata={"source": "local"})]


In [12]:
# ! pip install sentence_transformers -q

In [13]:
## 2. Embeddings

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings

if MODEL == "gemini":
        gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", api_key=GOOGLE_API_KEY)
else:
        embeddings=HuggingFaceEmbeddings(
                model_name='sentence-transformers/all-MiniLM-L6-v2', 
                model_kwargs={'device':'cpu'}
        )







c:\Users\LENOVO\sagar\work\AI_ML\gitrepo\GENAI\YOUTUBES\GenAI_research\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
## 3. Save Embeddings in vectorDB - chromadb

from langchain.vectorstores import Chroma

# Save to disk
vectorstore = Chroma.from_documents(
                     documents=documents,                 # Data
                     embedding=gemini_embeddings,    # Embedding model
                     persist_directory=CHROMA_VECTORDB_PATH # Directory to save data
                     )

### 4. Create a retriever

## Load the vectorDB data from disk
vectorstore_disk = Chroma(
                        persist_directory=CHROMA_VECTORDB_PATH,       # Directory of db
                        embedding_function=gemini_embeddings   # Embedding model
                   )

retriever_chromadb = vectorstore_disk.as_retriever(search_kwargs={"k": 3})

NameError: name 'gemini_embeddings' is not defined

In [ ]:
# print(len(retriever.get_relevant_documents("nvdia")))
retriever_documents = retriever_chromadb.get_relevant_documents("what is yolo?")
for doc in retriever_documents:
    print(doc.page_content)

c:\Users\LENOVO\sagar\work\AI_ML\gitrepo\GENAI\YOUTUBES\GenAI_research\venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


same between YOLO and Fast YOLO.
YOLO 57.9 77.0 67.2 57.7 38.3 22.7 68.3 55.9 81.4 36.2 60.8 48.5 77.2 72.3 71.3 63.5 28.9 52.2 54.8 73.9 50.8
Feature Edit [33] 56.3 74.6 69.1 54.4 39.1 33.1 65.2 62.7 69.7 30.8 56.0 44.6 70.0 64.4 71.1 60.2 33.3 61.3 46.4 61.7 57.8
R-CNN BB [13] 53.3 71.8 65.8 52.0 34.1 32.6 59.6 60.0 69.8 27.6 52.0 41.7 69.6 61.3 68.3 57.8 29.6 57.8 40.9 59.3 54.1
SDS [16] 50.7 69.7 58.4 48.5 28.3 28.8 61.3 57.5 70.8 24.1 50.7 35.9 64.9 59.1 65.8 57.1 26.0 58.8 38.6 58.9 50.7
sion of Fast R-CNN. Other versions of Fast R-CNN provide only
a small beneﬁt while YOLO provides a signiﬁcant performance
boost.


In [ ]:
# ! pip install faiss-cpu -q

In [ ]:
## 3. Save Embeddings in vectorDB - FAISS

vector_db_directory = FAISS_VECTORDB_PATH
from langchain.vectorstores import FAISS

def store_data_in_vectordb(documents, embeddings):
    try:
        current_vectordb = load_vectordb(vector_db_directory, embeddings)
        print('current_vectordb - ', current_vectordb)
    except Exception as e:
        print('Exception inside storing data in vector db => ', e)

    new_knowledge_base =FAISS.from_documents(documents, embeddings)
    print('new_knowledge_base - ', new_knowledge_base)

    # Saving the new vector DB
    new_knowledge_base.save_local(vector_db_directory)
    return new_knowledge_base


def load_vectordb(stored_directory, embeddings):
    loaded_vector_db = FAISS.load_local(stored_directory, embeddings)
    return loaded_vector_db

vector_db=store_data_in_vectordb(documents, gemini_embeddings)

retriever_faissdb = vector_db.as_retriever(search_kwargs={"k": 2})

Exception inside storing data in vector db =>  The de-serialization relies loading a pickle file. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine.You will need to set `allow_dangerous_deserialization` to `True` to enable deserialization. If you do this, make sure that you trust the source of the data. For example, if you are loading a file that you created, and no that no one else has modified the file, then this is safe to do. Do not set this to `True` if you are loading a file from an untrusted source (e.g., some random site on the internet.).
new_knowledge_base -  <langchain_community.vectorstores.faiss.FAISS object at 0x0000016B8A67FE10>


In [ ]:
import pandas as pd
def convert_vectordb_to_df(vectorDb):
    try:
        vector_dict = vectorDb.docstore._dict
        data_rows = []

        for k in vector_dict.keys():
            doc_name = vector_dict[k].metadata['source'].split('/')[-1]
            if 'page' not in vector_dict[k].metadata:
                page_number = 1
            else:
                page_number = vector_dict[k].metadata['page'] + 1
            content =  vector_dict[k].page_content
            data_rows.append({"chunk_id": k, "document": doc_name, "page": page_number, "content":content})

        vector_df = pd.DataFrame(data_rows)
        # print(vector_df)
        return vector_df
    except Exception as e :
        print('Error in convert_vectordb_to_df - ', e)
        return None
    

convert_vectordb_to_df(vector_db) # for faissdb

                                chunk_id             document  page  \
0   2f8f317b-e6e9-482b-909c-41aefb0892bf  ..\..\data\yolo.pdf     1   
1   c170bce6-16a5-4cdc-970d-a37f98aa9bba  ..\..\data\yolo.pdf     1   
2   0b6a928b-c232-4558-b36f-e7bb25acac31  ..\..\data\yolo.pdf     1   
3   f1d67a3e-c288-4e14-a1f0-ac025a64e86d  ..\..\data\yolo.pdf     1   
4   123ce14f-f86c-40fb-a4f8-413e55b3190c  ..\..\data\yolo.pdf     1   
..                                   ...                  ...   ...   
91  49756c4e-8713-49e0-a180-0833d77ca7a5  ..\..\data\yolo.pdf     9   
92  b3d8130e-ade3-411a-97a8-6ca48f76289f  ..\..\data\yolo.pdf     9   
93  3c3ec2b2-7a76-416a-9d8d-1a8de1a6cc7c  ..\..\data\yolo.pdf    10   
94  2bfc71a7-fcb0-404d-8bd2-cae05d4c6197  ..\..\data\yolo.pdf    10   
95  63caa967-b23b-42a5-8442-fab786ec1f3b  ..\..\data\yolo.pdf    10   

                                              content  
0   You Only Look Once:\nUniﬁed, Real-Time Object ...  
1   associated class probabilities.

,chunk_id,document,page,content
0,2f8f317b-e6e9-482b-909c-41aefb0892bf,..\..\data\yolo.pdf,1,"You Only Look Once:\nUniﬁed, Real-Time Object ..."
1,c170bce6-16a5-4cdc-970d-a37f98aa9bba,..\..\data\yolo.pdf,1,associated class probabilities. A single neura...
2,0b6a928b-c232-4558-b36f-e7bb25acac31,..\..\data\yolo.pdf,1,still achieving double the mAP of other real-t...
3,f1d67a3e-c288-4e14-a1f0-ac025a64e86d,..\..\data\yolo.pdf,1,"jects are in the image, where they are, and ho..."
4,123ce14f-f86c-40fb-a4f8-413e55b3190c,..\..\data\yolo.pdf,1,"for general purpose, responsive robotic system..."
...,...,...,...,...
91,49756c4e-8713-49e0-a180-0833d77ca7a5,..\..\data\yolo.pdf,9,"abs/1504.06066, 2015. 3, 7\n[30] O. Russakovsk..."
92,b3d8130e-ade3-411a-97a8-6ca48f76289f,..\..\data\yolo.pdf,9,and Y . LeCun. Overfeat: Integrated recognitio...
93,3c3ec2b2-7a76-416a-9d8d-1a8de1a6cc7c,..\..\data\yolo.pdf,10,[33] Z. Shen and X. Xue. Do more dropouts in p...
94,2bfc71a7-fcb0-404d-8bd2-cae05d4c6197,..\..\data\yolo.pdf,10,4\n[36] P. Viola and M. Jones. Robust real-tim...


In [ ]:
def get_similiar_docs(vector_db, query,k=3,score=False):
  
    vectordb_results = list()
    if score:
        similar_docs_with_score = vector_db.similarity_search_with_score(query,k=k)
        for doc, score in similar_docs_with_score:
            print(f"Content: {doc.page_content}, Metadata: {doc.metadata}, Score: {score}")
            vectordb_results.append(doc.page_content)
    else:
        similar_docs = vector_db.similarity_search(query,k=k)
        for doc in similar_docs:
            vectordb_results.append(doc.page_content)

    return vectordb_results


get_similiar_docs(vector_db, 'what is YOLO?')

['same between YOLO and Fast YOLO.',
 'YOLO 57.9 77.0 67.2 57.7 38.3 22.7 68.3 55.9 81.4 36.2 60.8 48.5 77.2 72.3 71.3 63.5 28.9 52.2 54.8 73.9 50.8\nFeature Edit [33] 56.3 74.6 69.1 54.4 39.1 33.1 65.2 62.7 69.7 30.8 56.0 44.6 70.0 64.4 71.1 60.2 33.3 61.3 46.4 61.7 57.8\nR-CNN BB [13] 53.3 71.8 65.8 52.0 34.1 32.6 59.6 60.0 69.8 27.6 52.0 41.7 69.6 61.3 68.3 57.8 29.6 57.8 40.9 59.3 54.1\nSDS [16] 50.7 69.7 58.4 48.5 28.3 28.8 61.3 57.5 70.8 24.1 50.7 35.9 64.9 59.1 65.8 57.1 26.0 58.8 38.6 58.9 50.7',
 'sion of Fast R-CNN. Other versions of Fast R-CNN provide only\na small beneﬁt while YOLO provides a signiﬁcant performance\nboost.']

In [ ]:
## GENERATOR


from langchain_google_genai import ChatGoogleGenerativeAI

# llm = ChatGoogleGenerativeAI(
#     model="gemini-pro",
#     temperature=0.1
# )

## Huggingface Model

from langchain import HuggingFaceHub

HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")
os.environ["HUGGINGFACEHUB_API_TOKEN"]
llm = HuggingFaceHub(repo_id="mistralai/Mistral-7B-Instruct-v0.2", huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN)


c:\Users\LENOVO\sagar\work\AI_ML\gitrepo\GENAI\YOUTUBES\GenAI_research\venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [ ]:
## prompt templates:

from langchain import PromptTemplate
from langchain.schema.prompt_template import format_document

llm_prompt_template = """You are an assistant for question-answering tasks.
Use the following context to answer the question.
If you don't know the answer, just say that you don't know.
Use five sentences maximum and keep the answer concise.\n
Question: {question} \nContext: {context} \nAnswer:"""


# llm_prompt_template = """You are an assistant for question-answering tasks. The documentation is located at SOURCE_DOCUMENTS.
# You are given the following extracted parts of a long document and a question. 
# You should only use source and data that are explicitly listed as a source in the context. 

# Do NOT use any external resource, hyperlink or reference to answer that is not listed.

# If you don't know the answer, just say "Hmm, I'm not sure." Don't try to make up an answer.
    
# {context}
    
# Question: {question}
# """

llm_prompt = PromptTemplate.from_template(llm_prompt_template)

print(llm_prompt)

input_variables=['context', 'question'] template="You are an assistant for question-answering tasks.\nUse the following context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse five sentences maximum and keep the answer concise.\n\nQuestion: {question} \nContext: {context} \nAnswer:"


In [ ]:
## choosing a retriever

# retriever = retriever_chromadb
retriever = retriever_faissdb

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser


# Combine data from documents to readable string format.
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Create stuff documents chain using LCEL.
# This is called a chain because you are chaining together different elements
# with the LLM. In the following example, to create the stuff chain, you will
# combine the relevant context from the website data matching the question, the
# LLM model, and the output parser together like a chain using LCEL.
#
# The chain implements the following pipeline:
# 1. Extract the website data relevant to the question from the Chroma
#    vector store and save it to the variable `context`.
# 2. `RunnablePassthrough` option to provide `question` when invoking
#    the chain.
# 3. The `context` and `question` are then passed to the prompt where they
#    are populated in the respective variables.
# 4. This prompt is then passed to the LLM (`gemini-pro`).
# 5. Output from the LLM is passed through an output parser
#    to structure the model's response.
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | llm_prompt
    | llm
    | StrOutputParser()
)

## inference
rag_chain.invoke("What is YOLO?")


'The provided context does not mention what YOLO is, so I cannot answer this question from the provided context.'

In [ ]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type='stuff',
            retriever=retriever,
            return_source_documents=False,
            chain_type_kwargs={'prompt': llm_prompt}
    )

result=chain({'query':'What is ai?'}, return_only_outputs=True)
print(result)

{'result': 'The provided context does not contain any information about what AI is, so I cannot answer this question from the provided context.'}
